# Fourier Picture from SVG file

Some remarks to start with:
You can create an SVG file (https://en.wikipedia.org/wiki/Scalable_Vector_Graphics) with many grafic programs. I used GIMP to draw the outline of my silhouette from a picture in profile. For a simple guide how to draw the path and how to export is as an SVG file in GIMP look for example here: https://www.useragentman.com/blog/2013/04/26/how-to-create-svg-paths-easily-using-the-gimp/

The SVG file stores the path in terms of path segments which consist of cubic Bezier curves. If you want to learn a bit about Bezier curves, here is a really nice YouTube video: https://www.youtube.com/watch?v=aVwxzDHniEw

With svgpathtools Python offers a really nice and extensive library taking care of almost everything one would like to do with SVG files. I also use matplotlib to generate the final animation. os is a library which helps you interacting with the operating system. I use it to check whether a file exists already and to delete it if necessary.

So, let's import the necessary libraries

In [ ]:
import numpy as np
from svgpathtools import svg2paths, wsvg
from svgpathtools.path import CubicBezier
import matplotlib.pyplot as pl 
from matplotlib import animation
import os

Now load your SVG file and pass it to svg2paths to extract the path segments (the attributes are not strictly necessary)

In [ ]:
pict = "/Path of your SVG/YourSVG.svg"

paths, attributes = svg2paths(pict)

Create a list for the points on the path "xes" a linear space to interpolate from 0 to 1 with 20 intermediate points. The rate gives determines the sampling rate. These parameters need a bit of experimenting to adjust them to the picture. 

In [3]:
xes = []
vals = np.linspace(0, 1, 20)
rate = 1

Now we extract the path segments and use them to build a set of points on the path in the complex plane.

In [ ]:
for path in paths[0]:
    
    # find third order polynomial, s.t. polynomial(0) = start of path segment and polynomial(1) = end of path segment
    # luckily .poly() does the trick for cubic Bezier curves :-)
    pathpolynomial = path.poly()
    
    # get 20 points in the complex plane along the path from the polynomial using the values from linspace
    points = pathpolynomial(vals)
    
    # use the clever numpy c_ function to build a list of the compex points representes as 2-element lists 
    # of real and imaginary part
    points = np.c_[points.real, points.imag]
    
    # approximate the length of each path by summing over the distances between the 20 base points
    length = points[1:] - points[:-1]
    length = int(np.sqrt((length * length).sum()))
    
    # build a linspace to reduce the number of basepoints to a managable amount using the "sampling" rate
    reducedpoints = np.linspace(0, 1, int(length / rate) + 1, endpoint=False)
    
    # collect the reduced number of basepoints into a list
    xes.extend(pathpolynomial(reducedpoints))
    
print(len(xes),xes[0],xes[len(xes)-1])
    

And we put the result into an array, shifting and rescaling a bit...

In [6]:
# build a plottable array from the points
xx = np.asarray(xes)
# shift to zero
xx -= xx.mean(0)
# scale to 1
scale = max(np.abs(xx.real).max(), np.abs(xx.imag).max())
xx /= scale
# image is upside down. So reflect on real axis
xx = xx.real - 1j * xx.imag

Let's see what we get in real space

In [ ]:
# let's have a look what we have...
pl.figure(figsize=(10, 10))
pl.scatter(xx.real, xx.imag,s=0.1)
pl.plot(xx.real, xx.imag, lw=0.5)
pl.axis('scaled')

Next, we use fast Fourier transform to get the (discrete) Fourier coefficients of the path approximation

In [8]:
# calculate and normalise fourier coefficients
Fxx    = np.fft.fft(xx) / xx.size
# calculate fourier frequencies
frequencies = np.fft.fftfreq(Fxx.size, 1/Fxx.size)
# calculate absolute values, i.e. radii of circles
radii = np.abs(Fxx)

In [ ]:
# just check the maximal radius
radii.max()

In [ ]:
# we probably have too many fourier coefficients
len(Fxx)

reduce the number of Fourier coefficients either with  
N = (rad_norm < 0.91).sum() + 1
to a given accuracy of just set it to a number of your choice. Playing around with this number is fun :-)

In [12]:
# first, get the list of indices of the entries in radii from biggest to smallest
inds = np.argsort(-radii)
# define a list with the normalised radii
rad_norm = radii / radii.sum()
# build a list with partial sums (cumsum) of the first n biggest entries
rad_norm = np.cumsum(rad_norm[inds])
# find the number N of Fourier coefficients, s.t. the partial sum is below a threshold
#N = (rad_norm < 0.91).sum() + 1
N = 50 

In [13]:
# now find the top N indeces 
top_inds = (inds[:N])
top_inds = top_inds[top_inds != 0]

In [14]:
# select the corresponding fourier coefficients, radii and frequencies
Fxx_sel = Fxx[top_inds]
radii_sel = radii[top_inds]
freq_sel = frequencies[top_inds]

In [16]:
# fix the center of the first fourier epicycle
#base = Fxx_sel[0]
# time points
#time = list(enumerate(np.linspace(0, 2 * np.pi, 100)))
# linspace for points on the fourier circles
theta = np.linspace(-np.pi, np.pi, N)
points = []

Build a function plotting at given time t one epicycle image. Iterating over all time points produces pictures for the animation (and the gif).

In [ ]:
def plotfig(t):
    base = Fxx_sel[0]
    pl.clf()
    #pl.plot(xx.real, xx.imag, lw=0.5)
    for count, (Fxx_i, rad_i, freq_i) in enumerate(zip(Fxx_sel, radii_sel, freq_sel)):
        # find point on epicycle at time t
        r_i = Fxx_i * np.exp(1j * freq_i * t) 
        # draw center of epicycle
        pl.scatter([base.real], [base.imag], c= 'blue', s=0.1,lw=0.1)
        # build list with points on circle of epicycle 
        xy = np.c_[np.cos(theta) * rad_i + base.real, np.sin(theta) * rad_i + base.imag]
        # plot circle
        pl.plot(xy[:, 0], xy[:, 1], c= 'blue', lw=0.1)
        # plot line from center of circle to r_i
        pl.plot([base.real, base.real + r_i.real], [base.imag, base.imag + r_i.imag], c='blue', lw=0.1)
        # shift to center of next epicycle
        base += r_i
        pl.xticks([])
        pl.yticks([])
        pl.xlim((-2, 2))
        pl.ylim((-2, 2))
        pl.axis('off')

    points.append(base)
    pl.scatter([base.real], [base.imag], c='red')
    pp = np.asarray(points)
    pl.plot(pp.real, pp.imag, lw=0.5)
    

And we finally build the gif. Be careful with the number of frames n_frames as this can take a while...

In [ ]:
gifname = '/Path for your final gif/nameGif.gif'

n_frames = 200 

fig, ax = pl.subplots(1, 1, figsize=(5, 5))

anim = animation.FuncAnimation(
        fig=fig,    
        func=plotfig, 
        frames=np.linspace(0, 4 * np.pi, n_frames), 
        interval=20,
    )

if os.path.isfile(gifname):
   os.remove(gifname)

anim.save(gifname, writer='imagemagick',fps = 20)

pl.show()
pl.clf()
pl.close()

And that's basically it. The gif-Logo of myself has been created exactly like this!